In [17]:
library(Rdisop)
source("code/matchIonizationMode.R")
source("code/get_form.R")
source("code/split_and_filter.R")

In [1]:
# import atom exact mass to adduct calculation
adduct_n <- read.csv("ESI-MS-adducts-calculator-Vsept2012_neg.csv")
adduct_p <- read.csv("ESI-MS-adducts-calculator-Vsept2012_pos.csv")
head(adduct_n)
head(adduct_p)

Ion.name,Ion.mass,Charge,Mult,Mass
M-3H,M/3 - 1.007276,3-,0.3333333,-1.007276
M-2H,M/2 - 1.007276,2-,0.5000000,-1.007276
M-H2O-H,M- 19.01839,1-,1.0000000,-19.018390
M-H,M - 1.007276,1-,1.0000000,-1.007276
M+Na-2H,M + 20.974666,1-,1.0000000,20.974666
M+Cl,M + 34.969402,1-,1.0000000,34.969402


Ion.name,Ion.mass,Charge,Mult,Mass
M+3H,M/3 + 1.007276,3+,0.33,1.007276
M+2H+Na,M/3 + 8.334590,3+,0.33,8.334590
M+H+2Na,M/3 + 15.7661904,3+,0.33,15.766190
M+3Na,M/3 + 22.989218,3+,0.33,22.989218
M+2H,M/2 + 1.007276,2+,0.50,1.007276
M+H+NH4,M/2 + 9.520550,2+,0.50,9.520550


In [2]:
# assign masses to the adducts of interest
na <- 22.989218
h <- 1.007276
cl <- 34.969402

In [3]:
# Load the feature tables from positive and negative mode
# exclude the line with the *60 if the retention time is already
# in seconds
pos <- read.csv("TEST1/161205_Positive_filtered_shared_features.csv")
pos <- pos[, c("row.m.z", "row.retention.time")]
pos[,2] <- pos[,2]*60

neg <- read.csv("TEST1/12042016_filtered_Negative2.csv")
neg <- neg[, c("row.m.z", "row.retention.time")]
neg[,2] <- neg[,2]*60

In [6]:
# adductdiff = 2 * h = 2.014552 and  mz.abs.tol (+/- 0.005) accounts for absolute m/z error
# rt.abs.tol=20 is the allowed retention time drift between aquisition modes
negx <- matchIonizationMode(neg[,c("row.m.z", "row.retention.time")], pos[,c("row.m.z", "row.retention.time")], adductdiff=2.014552, mz.abs.tol=0.005, rt.abs.tol=20)

In [7]:
# adductdiff = na + h = 23.99649 and  mz.abs.tol (+/- 0.01) accounts for absolute m/z error
# rt.abs.tol=20 is the allowed retention time drift between aquisition modes
negxNa <- matchIonizationMode(neg[,c("row.m.z", "row.retention.time")], pos[,c("row.m.z", "row.retention.time")], adductdiff=23.99649, mz.abs.tol=0.01, rt.abs.tol=20)

In [8]:
# adductdiff = -(cl - h) = -33.96213, here the difference has to be negative, because the mass of 
# [M+Cl] > [M+H]
# mz.abs.tol (+/- 0.01) accounts for absolute m/z error
# rt.abs.tol=20 is the allowed retention time drift between aquisition modes
negxCl <- matchIonizationMode(neg[,c("row.m.z", "row.retention.time")], pos[,c("row.m.z", "row.retention.time")], adductdiff=-33.96213, mz.abs.tol=0.01, rt.abs.tol=20)

In [9]:
# Compile the tables and generate a table with only ions matching positive and negative mode
allneg <- cbind(negx, negxNa[,-c(1,2)])
allneg <- cbind(allneg, negxCl[,-c(1,2)])
colnames(allneg)[3:8] <- c( "mass_on_pos_tableH", "rt_on_pos_tableH", "mass_on_pos_tableNa", "rt_on_pos_tableNa", "mass_on_pos_tableCl", "rt_on_pos_tableCl")
ex <- apply(allneg, 1, function(x) sum(x[3:8]=="")==6)
allnegs <- allneg[!ex,]

In [10]:
dim(neg)

[1] 8546    2

In [11]:
head(allnegs)

,row.m.z,row.retention.time,mass_on_pos_tableH,rt_on_pos_tableH,mass_on_pos_tableNa,rt_on_pos_tableNa,mass_on_pos_tableCl,rt_on_pos_tableCl
3,265.1478,604.9176,267.1593434;267.1591933,607.786818;592.86199998,289.1538153;289.141276,613.5690486;600.0843336,,
4,331.2491,535.4694,333.2637768,535.2827625,355.246215,535.27910256,,
7,299.2591,641.8074,,,323.2580719;323.2580414,659.7529998;623.3655,,
8,221.1546,617.2219,223.1697655,617.3025,,,,
10,233.1546,618.5047,235.1695874;235.1696329,618.321198;605.569875,,,,
11,297.1528,606.4535,299.1648356,605.8429998,,,263.1980866,623.9719554


In [14]:
flist <- list()
flags <- c()
for(i in 1:nrow(allnegs)) {
#for(i in 1:5) {        
        posh <- c()
        negh <- c()
        negcl <- c()
        posna <- c()

        if(sum(allnegs[i, c(3,5)]=="")==2 & allnegs[i, 7]!="") {
                negcl <- get_form(as.numeric(strsplit(as.character(allnegs[i,1]), ";")[[1]])[1]-34.969402)
        } else {
                negh <- get_form(as.numeric(allnegs[i, 1])+1.007276)
                if(allnegs[1, 7]!="") {
                        negcl <- get_form(as.numeric(strsplit(as.character(allnegs[i,1]), ";")[[1]])[1]-34.969402)
                }
        }

        if(allnegs[i, "mass_on_pos_tableH"]!="") {
                posh <- get_form(as.numeric(strsplit(as.character(allnegs[i, "mass_on_pos_tableH"]), ";")[[1]])[1]-1.007276)
        }
        if(allnegs[i, "mass_on_pos_tableNa"]!="") {
                posna <- get_form(as.numeric(strsplit(as.character(allnegs[i, "mass_on_pos_tableNa"]), ";")[[1]])[1]-22.989218)
        }
        flag <- ""

        if(length(negh)>0 | length(negcl)>0 ) {
                if(length(negh)==0 | length(negcl)>0 ) {
                        negh <- negcl
                        flag <- "+cl"
                } else {
                        flag <- "-h"
                }

                if(length(posh)>0 & sum(negh[,1] %in% posh[,1]) ) {
                        negh <- matrix(negh[negh[,1] %in% posh[,1], ], ncol=2)
                        flag <- paste0(flag, "+h")
                }

                if(length(posna)>0 & sum(negh[,1] %in% posna[,1]) ) {
                        negh <- matrix(negh[negh[,1] %in% posna[,1], ], ncol=2)
                        flag <- paste0(flag, "+na")
                }

                if(length(negcl)>0 & sum(negh[,1] %in% negcl[,1]) ) {
                        negh <- matrix(negh[negh[,1] %in% negcl[,1], ], ncol=2)
                        flag <- paste0(flag, "+cl")
                }
        }
        flist[[i]] <- negh
        flags <- c(flags, flag)
}



In [15]:
# Format the table to keep the 10 first can. It can allow more changing the number 10 below
# Remember that this formulas were generated with ppm=5/without P 
for(i in 1:length(flist)) if(!is.null(flist[[i]])) { if(nrow(flist[[i]])>10) flist[[i]] <- flist[[i]][1:10,] } else { flist[[i]] <- matrix("", ncol=2) }

fms <- lapply(flist, function(x) paste(x[,1], collapse=";"))
sc <- lapply(flist, function(x) paste(x[,2], collapse=";"))
allnegs <- data.frame(allnegs, Formulas=unlist(fms), Scores=unlist(sc), Flag=flags)

In [16]:
allnegs

,row.m.z,row.retention.time,mass_on_pos_tableH,rt_on_pos_tableH,mass_on_pos_tableNa,rt_on_pos_tableNa,mass_on_pos_tableCl,rt_on_pos_tableCl,Formulas,Scores,Flag
3,265.1478,604.9176,267.1593434;267.1591933,607.786818;592.86199998,289.1538153;289.141276,613.5690486;600.0843336,,,C12H26O4S;C4H22N6O7;C5H34N2OS4;C4H30N2O6S2;C5H147N3O;C5H26N6O2S2;C5H18N10O3;C3H26N2O11;CH14N16O,0.291728046899418;0.265637906597544;0.115116666366116;0.0178988621545199;0.00680963298497408;0.00319911620256948;8.32079874865639e-06;2.59788454699252e-07;2.14836997744768e-07,-h
4,331.2491,535.4694,333.2637768,535.2827625,355.246215,535.27910256,,,C3H40N8O5S2;C18H36O5;C11H44N2O2S3;C4H157N9;C3H32N12O6;C3H48N4O4S4;C11H36N6O3S;C10H40N2O7S;C19H40S2;C4H44N8S4,0.113609629123458;0.103916652331871;0.103796778991732;0.0825482311231669;0.0375946782120292;0.0182239611311646;0.0151462362298204;0.00492264716599492;0.00490877721265675;0.00389623355815448,-h+h+na
7,299.2591,641.8074,,,323.2580719;323.2580414,659.7529998;623.3655,,,H173NOS3,5.91018600546938e-07,-h+na
8,221.1546,617.2219,223.1697655,617.3025,,,,,C14H22O2;C6H26N2O4S;C7H22N6S,0.309136101869248;0.0119242665122486;0.000503735287817614,-h+h
10,233.1546,618.5047,235.1695874;235.1696329,618.321198;605.569875,,,,,C15H22O2;H26N8O2S2;H34N4OS4;C7H26N2O4S;H18N12O3;H42S6;C8H22N6S,0.220536182010116;0.188597561587354;0.0829253590813214;0.0146950878027628;0.00487032952530153;0.000874469717409341;0.000521971565053549,-h+h
11,297.1528,606.4535,299.1648356,605.8429998,,,263.1980866,623.9719554,H34N4O7S3;CH151N5O2S;C7H26N2O10;C5H14N16,7.162113892749e-05;1.8554956048685e-05;4.69206995154508e-06;4.0566511133493e-06,-h+h
12,242.1762,445.2437,244.1909996,445.07591142,266.1727686,445.04526084,,,C13H25NO3;C6H33N3S3;C6H25N7OS;C5H29N3O5S;C2H154O4,0.390273094575672;0.291457751717015;0.0104115640571638;0.00318771488247309;3.22655646480777e-06,-h+h+na
13,309.1742,638.6725,,,,,275.2036816;275.2217331;275.2214966,628.247238;650.9835;635.9290002,C2H22N14O2;C4H34O12;C2H30N10OS2;C17H26N2O;C2H38N6S4;C9H30N4O3S;C6H155NO2;C5H38O7S2;CH26N10O6;C9H38O2S3,0.167360230747657;0.159885607320987;0.122764683725015;0.103917178730147;0.0045790787339567;0.000479907506648241;0.000409650450851879;0.000104430060202324;4.87901991412277e-05;6.71886726877468e-07,+cl+cl
14,293.1759,577.6112,295.1902665;295.189859;295.189785,577.29411762;558.82639998;591.02575002,317.1752319;317.1743774,569.90266668;587.52466668,,,C18H22N4;C3H26N12S2,3.37295532309679e-05;2.2047299460728e-05,-h+h+na
15,293.1760,545.5763,295.189859;295.1903168,558.82639998;532.65379998,,,,,C10H34N2OS3;C2H30N8O4S2;C17H26O4;C2H22N12O5;C10H26N6O2S;C2H38N4O3S4;C9H30N2O6S;CH26N8O9;C2H46O2S6;CH34N4O8S2,0.191455990714109;0.158941186553322;0.141826512197692;0.0631331390885773;0.0223281531515091;0.0148943312825743;0.00270898187166745;0.000465204555454809;0.0002200861223127;1.26681270743918e-06,-h+h


In [ ]:
# Output table
write.table(allnegs, "match_adduct_H_and_Na_Cl_with_formulas.tsv", row.names=FALSE, sep="\t")

In [18]:
# Filter formulas and replace in the old matrix
allnegs[,"Scores"] <- as.character(allnegs[,"Scores"])
allnegs[,"Formulas"] <- as.character(allnegs[,"Formulas"])
allnegs[,"Scores"] <- sapply(1:nrow(allnegs), function(x) paste(strsplit(as.character(allnegs[x,"Scores"]), ";")[[1]][unlist(lapply(lapply(strsplit(as.character(allnegs[x,"Formulas"]), ";")[[1]], split.fm) , filter.rules) )], collapse=";"))
allnegs[,"Formulas"] <- sapply(allnegs[,"Formulas"], function(x) paste(strsplit(x, ";")[[1]][unlist(lapply(lapply(strsplit(x, ";")[[1]], split.fm) , filter.rules) )], collapse=";") )

In [19]:
allnegs

,row.m.z,row.retention.time,mass_on_pos_tableH,rt_on_pos_tableH,mass_on_pos_tableNa,rt_on_pos_tableNa,mass_on_pos_tableCl,rt_on_pos_tableCl,Formulas,Scores,Flag
3,265.1478,604.9176,267.1593434;267.1591933,607.786818;592.86199998,289.1538153;289.141276,613.5690486;600.0843336,,,C12H26O4S;C5H18N10O3,0.291728046899418;8.32079874865639e-06,-h
4,331.2491,535.4694,333.2637768,535.2827625,355.246215,535.27910256,,,C18H36O5;C11H36N6O3S;C10H40N2O7S;C19H40S2,0.103916652331871;0.0151462362298204;0.00492264716599492;0.00490877721265675,-h+h+na
7,299.2591,641.8074,,,323.2580719;323.2580414,659.7529998;623.3655,,,,,-h+na
8,221.1546,617.2219,223.1697655,617.3025,,,,,C14H22O2;C7H22N6S,0.309136101869248;0.000503735287817614,-h+h
10,233.1546,618.5047,235.1695874;235.1696329,618.321198;605.569875,,,,,C15H22O2;C7H26N2O4S;C8H22N6S,0.220536182010116;0.0146950878027628;0.000521971565053549,-h+h
11,297.1528,606.4535,299.1648356,605.8429998,,,263.1980866,623.9719554,,,-h+h
12,242.1762,445.2437,244.1909996,445.07591142,266.1727686,445.04526084,,,C13H25NO3,0.390273094575672,-h+h+na
13,309.1742,638.6725,,,,,275.2036816;275.2217331;275.2214966,628.247238;650.9835;635.9290002,C17H26N2O;C9H30N4O3S,0.103917178730147;0.000479907506648241,+cl+cl
14,293.1759,577.6112,295.1902665;295.189859;295.189785,577.29411762;558.82639998;591.02575002,317.1752319;317.1743774,569.90266668;587.52466668,,,C18H22N4,3.37295532309679e-05,-h+h+na
15,293.1760,545.5763,295.189859;295.1903168,558.82639998;532.65379998,,,,,C17H26O4;C10H26N6O2S;C9H30N2O6S,0.141826512197692;0.0223281531515091;0.00270898187166745,-h+h


In [20]:
# split the first ranking formula in its atoms
lf1 <- sapply(allnegs[,"Formulas"], function(x) strsplit(x, ";")[[1]][1])

dfl <- lapply(lapply(lf1[!is.na(lf1)], split.fm), get.df)
log <- lapply(1:length(dfl), function(x) dfl[[x]]$id <<- x)
df1 <- dfl[[1]]
for(i in 2:length(dfl)) {
        df1 <- merge(x = df1, y = dfl[[i]],  all = TRUE)
}

df1 <- df1[order(df1$id), c("id", "C", "H", "O", "N", "S")]
df1[is.na(df1)] <- 0
fms5ppmNoP <- data.frame(allnegs[!is.na(lf1),1:2 ], lf1[!is.na(lf1)], df1)
colnames(fms5ppmNoP)[3] <- "FirstRankForm"


In [21]:
head(fms5ppmNoP)

,row.m.z,row.retention.time,FirstRankForm,id,C,H,O,N,S
3,265.1478,604.9176,C12H26O4S,1,12,26,4,0,1
4,331.2491,535.4694,C18H36O5,2,18,36,5,0,0
8,221.1546,617.2219,C14H22O2,3,14,22,2,0,0
10,233.1546,618.5047,C15H22O2,4,15,22,2,0,0
12,242.1762,445.2437,C13H25NO3,5,13,25,3,1,0
13,309.1742,638.6725,C17H26N2O,6,17,26,1,2,0


In [ ]:
write.table(fms3ppmNoP, "fms3ppmNoP.tsv", row.names=FALSE, sep="\t")

## Come here only for neg

In [24]:
# Load data and set variables
library(Rdisop)
source("code/matchIonizationMode.R")
source("code/get_form.R")
source("code/split_and_filter.R")
h <- 1.007276
cl <- 34.969402

# In case you want to calculate sum formulas with ions that do not match to positive
# If you have only negative mode data you can simple do:
negx <- neg

#negx <- neg[!((as.character(neg[,1]) %in% as.character(allnegs[,1])) & (as.character(neg[,2]) %in% as.character(allnegs[,2]))),]

In [25]:
# replace old method
get.df <- function(fml, idx) as.data.frame(matrix(c(idx, fml[[2]]), dimnames=list(1, c("idx", fml[[1]])), nrow=1))

In [26]:
# Notice that we are only doing [M-H] and using the h variable declared in the
# beginning of the code
f5ppmHnoP <- c("", "")
lf5ppmHnoP <- list()
for(i in 1:nrow(negx)) {
        tmp <- get_form(negx[i,1]+h, ppm=5, elements = initializeElements(c("C","H", "N", "O", "S")))
        if(nrow(tmp)>0) {
                if(nrow(tmp)>10) {
                        tmp <- tmp[1:10,]
                }
                idx <- sapply(tmp[,1], function(x) filter.rules(split.fm(x)))
                f5ppmHnoP <- rbind(f5ppmHnoP, c(paste(tmp[idx,1], collapse=","),  paste(tmp[idx,2], collapse=",")))
                if(sum(idx)) {
                        lf5ppmHnoP[[i]] <- get.df(split.fm(tmp[idx,1][1]), i)
                }
        } else {
                f5ppmHnoP <- rbind(f5ppmHnoP, c("", ""))
        }       
             
}       


In [27]:
negx <-cbind(negx[,1:2], f5ppmHnoP[-1,])
colnames(negx)[3:4] <- c("Formulas","Scores")
head(negx)

row.m.z,row.retention.time,Formulas,Scores
355.1581,686.2329,"C10H24N6O8,C10H32N2O7S2,C18H28O5S","0.089699770643745,0.0524623294097015,0.0503164497053307"
194.0821,583.3693,C10H13NO3,0.786343249167303
265.1478,604.9176,"C12H26O4S,C5H18N10O3","0.291728046899418,8.32079874865639e-06"
331.2491,535.4694,"C18H36O5,C11H36N6O3S,C10H40N2O7S,C19H40S2","0.103916652331871,0.0151462362298204,0.00492264716599492,0.00490877721265675"
250.1448,637.5046,"C14H21NO3,C7H21N7OS","0.289636695142152,0.0025814842328478"
293.1792,677.7425,"C14H30O4S,C7H22N10O3,C15H26N4S","0.191817160686449,0.000161707513069561,1.74226007868597e-05"


In [28]:
# Again, split formulas and format atom table
dfl <- lf5ppmHnoP
df1 <- data.frame()
for(i in 1:length(dfl)) {
        if(!is.null(dfl[[i]])) {
                if(length(df1)==0) {
                        df1 <- dfl[[i]]
                } else {
                        df1 <- merge(x = df1, y = dfl[[i]],  all = TRUE)
                }
        }
}

df1 <- df1[order(df1$id), c("idx", "C", "H", "O", "N", "S")]
df1[is.na(df1)] <- 0
fms5ppmHnoP <- data.frame(negx[df1$idx,1:2 ],  df1)

In [29]:
head(fms5ppmHnoP)

row.m.z,row.retention.time,idx,C,H,O,N,S
355.1581,686.2329,1,10,24,8,6,0
194.0821,583.3693,2,10,13,3,1,0
265.1478,604.9176,3,12,26,4,0,1
331.2491,535.4694,4,18,36,5,0,0
250.1448,637.5046,5,14,21,3,1,0
293.1792,677.7425,6,14,30,4,0,1


In [55]:
# Output table
write.table(negx, "DOM_Neg_all_filtered.tsv", row.names=FALSE, sep="\t")
write.table(fms5ppmHnoP, "DOM_Neg_Top1_atom.tsv", row.names=FALSE, sep="\t")

## For Pos only start here


In [35]:
# Load data and set variables
library(Rdisop)
source("code/matchIonizationMode.R")
source("code/get_form.R")
source("code/split_and_filter.R")
h <- 1.007276
na <- 22.989218

# In case you want to calculate sum formulas with ions that do not match to negative
# If you have only positive mode data you can simple do:
posx <- pos

#posx <- pos[!((as.character(pos[,1]) %in% as.character(mzs2)) & (as.character(pos[,2]) %in% as.character(rts2))),]

In [36]:
# replace old method
get.df <- function(fml, idx) as.data.frame(matrix(c(idx, fml[[2]]), dimnames=list(1, c("idx", fml[[1]])), nrow=1))

In [37]:
# Notice that we are only doing [M+H] and using the h variable declared in the
# beginning of the code
f5ppmHnoP <- c("", "")
lf5ppmHnoP <- list()

for(i in 1:nrow(posx)) {
        tmp <- get_form(posx[i,1]-h, ppm=1, elements = initializeElements(c("C","H", "N", "O", "S")))
        if(!is.null(tmp)) {
                if(nrow(tmp)!=0) {
                        if(nrow(tmp)>10) {
                                tmp <- tmp[1:10,]
                        }
                        idx <- sapply(tmp[,1], function(x) filter.rules(split.fm(x)))
                        f5ppmHnoP <- rbind(f5ppmHnoP, c(paste(tmp[idx,1], collapse=","),  paste(tmp[idx,2], collapse=",")))
                        if(sum(idx)) {
                                lf5ppmHnoP[[i]] <- get.df(split.fm(tmp[idx,1][1]), i)
                        }
                } else {
                        f5ppmHnoP <- rbind(f5ppmHnoP, c("", ""))
                }
        } else {
                f5ppmHnoP <- rbind(f5ppmHnoP, c("", ""))
        }
}

In [38]:
head(posx)

row.m.z,row.retention.time
531.4083,671.3660
553.3899,670.8391
1083.7911,671.3786
1061.8108,671.4954
242.2845,606.0859
437.1940,579.7484


In [39]:
posx <-cbind(posx[,1:2], f5ppmHnoP[-1,])
colnames(posx)[3:4] <- c("Formulas","Scores")
head(posx)

row.m.z,row.retention.time,Formulas,Scores
531.4083,671.3660,C23H58N6O3S2,0.0228580564848555
553.3899,670.8391,"C15H60N4O14S,C36H48N4O","0.0205634510885497,0.0183676819727887"
1083.7911,671.3786,C74H102N2O4,0.000875830838165673
1061.8108,671.4954,,
242.2845,606.0859,C16H35N,0.126053758929266
437.1940,579.7484,,


In [40]:
# Again, split formulas and format atom table
dfl <- lf5ppmHnoP
df1 <- data.frame()
for(i in 1:length(dfl)) {
        if(!is.null(dfl[[i]])) {
                if(length(df1)==0) {
                        df1 <- dfl[[i]]
                } else {
                        df1 <- merge(x = df1, y = dfl[[i]],  all = TRUE)
                }
        }
}

df1 <- df1[order(df1$id), c("idx", "C", "H", "O", "N", "S")]
df1[is.na(df1)] <- 0
fms5ppmHnoP <- data.frame(posx[df1$idx,1:2 ],  df1)

In [60]:
head(fms5ppmHnoP)

,MZ,RT,idx,C,H,O,N,S
2,547.2610,394.5784,2,23,38,11,4,0
4,427.2905,330.7627,4,13,42,7,6,1
5,369.2487,291.1573,5,10,36,6,6,1
6,361.2225,630.3100,6,11,32,5,6,1
8,267.1801,218.7413,8,12,26,6,0,0
10,627.2851,448.8458,10,19,50,18,2,1


In [88]:
# Output table
write.table(posx, "DOM_Pos_all.tsv", row.names=FALSE, sep="\t")
write.table(fms5ppmHnoP, "DOM_Pos_Top-1_atom.tsv", row.names=FALSE, sep="\t")